### PromptTemplate 
* [PromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html#langchain_core.prompts.prompt.PromptTemplate)
* [ChatPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain_core.prompts.chat.ChatPromptTemplate)
* [ChatMessagePromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatMessagePromptTemplate.html#langchain_core.prompts.chat.ChatMessagePromptTemplate)
* [FewShotPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.few_shot.FewShotPromptTemplate.html#langchain_core.prompts.few_shot.FewShotPromptTemplate)
* PartialPrompt

In [ ]:
# poetry add python-dotenv langchain langchain-openai

In [3]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_o


##### 1) PromptTemplate 의 from_template() 함수 사용
* 주로 LLM(텍스트 완성형 모델, ex. Ollama, GPT-3.5)과 함께 사용
* 하나의 문자열 프롬프트를 생성

In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from pprint import pprint

template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    # model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    model="openai/gpt-oss-120b",
    temperature=0.7
)

chain = prompt_template | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3})
pprint(response)

('ChatGPT는 대규모 텍스트 데이터를 이용해 언어의 통계적 패턴을 학습합니다. 이를 위해 트랜스포머 구조의 신경망을 사용해 문맥을 '
 '고려한 토큰 예측 작업을 수행하고, 손실 함수를 최소화하도록 파라미터를 최적화합니다. 이렇게 사전학습된 모델에 인간 피드백을 활용한 '
 '강화학습(RLHF)을 추가해 대화 품질과 안전성을 높입니다.')


##### 2) PromptTemplate 결합하기
* 동일한 Prompt 패턴을 사용하지만 여러 개의 질문을 작성해서 LLM을 실행할 수도 있습니다.

In [10]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
# combined_prompt.format(model_name="ChatGPT", count=3, language="영어")
print(combined_prompt)

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"영어"})

pprint(response)

input_variables=['count', 'language', 'model_name'] input_types={} partial_variables={} template='{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요.\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요.'
('**ChatGPT 모델의 학습 원리 (3문장)**  \n'
 '1. 대규모 텍스트 데이터를 사용해 **자기지도학습**(self‑supervised learning) 방식으로 사전 '
 '학습(pre‑training)을 수행합니다.  \n'
 '2. 문맥을 예측하는 **마스크드 언어 모델링**(masked language modeling)과 **다음 토큰 '
 '예측**(next‑token prediction) 목표를 통해 언어의 구조와 의미를 내부에 압축합니다.  \n'
 '3. 이후 특정 작업(예: 질문‑답변, 번역 등)에 맞게 **지도학습**(supervised fine‑tuning)이나 '
 '**강화학습**(RLHF, Reinforcement Learning from Human Feedback)으로 미세 조정하여 실용성을 '
 '높입니다.  \n'
 '\n'
 '---\n'
 '\n'
 '### ChatGPT 모델의 장점 요약  \n'
 '- **다양한 언어와 주제에 대한 높은 이해도**: 방대한 텍스트 코퍼스로부터 일반적인 지식과 언어 규칙을 학습해, 여러 분야의 질문에 '
 '자연스럽게 답변할 수 있습니다.  \n'
 '- **문맥 유지와 연속적인 대화**: 트랜스포머 기반의 어텐션 메커니즘 덕분에 이전 발화들을 기억하고 일관된 흐름의 대화를 이어갈 수 '
 '있습니다.  \n'
 '- **사용자 피드백을 통한 지속적 개선**: RLHF와 같은 방법으로 인간 피드백을 반영해 안전성, 정확성,

#### PromptTemplate 의 파라미터를 배열 형태로 하여 여러개 사용하는 경우

In [12]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 2},
    {"model_name": "Gemma", "count": 3},
    {"model_name": "Gemini", "count": 4},
    {"model_name": "Claude", "count": 5},
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions]
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

for prompt in formatted_prompts:
    print(type(prompt), prompt)
    response = llm.invoke(prompt)
    pprint(response.content)

['GPT-4 모델의 학습 원리를 2 문장으로 한국어로 답변해 주세요.', 'Gemma 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.', 'Gemini 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.', 'Claude 모델의 학습 원리를 5 문장으로 한국어로 답변해 주세요.']
<class 'str'> GPT-4 모델의 학습 원리를 2 문장으로 한국어로 답변해 주세요.
('GPT‑4는 대량의 텍스트 데이터를 사용해 다음에 올 단어를 예측하도록 스스로 학습하는 자기지도 학습 방식을 적용합니다. 이렇게 얻은 '
 '파라미터들을 미세조정(fine‑tuning)하거나 인간 피드백을 활용한 강화학습(RLHF)으로 구체적인 작업 수행 능력을 향상시킵니다.')
<class 'str'> Gemma 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.
('Gemma 모델은 대규모 텍스트 데이터를 사전 학습(pre‑training)하여 언어의 통계적 패턴과 의미 관계를 학습합니다. 이후, '
 '특정 작업에 맞게 소량의 라벨링된 데이터로 미세 조정(fine‑tuning)함으로써 질문 답변, 요약 등 다양한 응용에 최적화됩니다. '
 '학습 과정에서는 트랜스포머 아키텍처와 자기‑주의(self‑attention) 메커니즘을 활용해 문맥 정보를 효과적으로 통합합니다.')
<class 'str'> Gemini 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('Gemini 모델은 대규모 텍스트와 멀티모달 데이터를 사용해 사전 학습(pre‑training) 단계에서 언어와 이미지 등 다양한 입력 '
 '형태의 패턴을 스스로 발견하도록 훈련됩니다.  \n'
 '이후에는 인간 피드백을 활용한 강화 학습(RLHF) 과정을 거쳐, 모델이 생성하는 답변의 정확성·유용성·안전성을 높입니다.  \n'
 '학습 과정에서 트랜스포머 기반의 어텐션 메커니즘을 활용해 문맥을 장기적으로 이해하고, 여러 모달 간의 연관성을 효율적으로 '
 '통합합니다.  \n'
 '마지막으로 

##### 2) ChatPromptTemplate
* Tuple 형태의 system, user, assistant 메시지 지원
* 여러 개의 메시지를 조합하여 LLM에게 전달 가능
* 간결성과 가독성이 높고 단순한 구조

In [13]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    # role, message
    ("system", "This system is an expert in answering questions about {topic}. Please provide clear and detailed explanations."),
    ("human", "{model_name} 모델의 학습 원리를 설명해 주세요."),
])

messages = chat_prompt.format_messages(topic="AI", model_name="ChatGPT")
print(messages)

# 생성한 메시지를 바로 주입하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

print(type(response))
print(response.content)

[SystemMessage(content='This system is an expert in answering questions about AI. Please provide clear and detailed explanations.', additional_kwargs={}, response_metadata={}), HumanMessage(content='ChatGPT 모델의 학습 원리를 설명해 주세요.', additional_kwargs={}, response_metadata={})]
<class 'langchain_core.messages.ai.AIMessage'>
## ChatGPT 모델이 어떻게 학습되는지 (한국어로)  

아래에서는 **ChatGPT**(또는 일반적인 대형 언어 모델, LLM)가 학습되는 전 과정을 크게 **세 단계**로 나눠서 설명합니다.

---

### 1️⃣ 사전 학습 (Pre‑training) – “대량 텍스트를 읽는 단계”

| 단계 | 핵심 내용 | 왜 필요한가? |
|------|-----------|--------------|
| **데이터 수집** | 웹 페이지, 책, 논문, 위키피디아, 뉴스, 포럼 등 수십 ~ 수백 TB 규모의 텍스트를 크롤링·정제합니다. 개인정보·유해 콘텐츠는 필터링하고, 언어·도메인 다양성을 확보합니다. | 모델이 **다양한 상황·주제**에 대해 일반적인 언어 패턴을 배우게 함. |
| **토큰화 (Tokenization)** | 텍스트를 **Byte‑Pair Encoding (BPE)** 혹은 **SentencePiece** 같은 서브워드 토크나이저로 변환합니다. 예: “ChatGPT” → `['Chat', '##G', '##PT']`. | 어휘 크기를 적당히 유지하면서 **희소성**을 줄이고, 새로운 단어도 처리 가능하게 함. |
| **목표 함수** | **다음 토큰 예측 (Next‑Token Prediction)**, 즉 **자기 회귀 언어 모델링**: \(\displaystyle \max

In [14]:
# 체인을 생성하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain = chat_prompt | llm | StrOutputParser()

response = chain.invoke({"topic":"AI", "model_name":"ChatGPT"})
print(type(response))
print(response)

<class 'str'>
## ChatGPT 모델이 어떻게 학습되는지  
(한국어로 상세히 설명합니다)

---

### 1. 전체 흐름 개요
1. **데이터 수집 & 전처리** → 2. **토크나이징** → 3. **대규모 사전 학습 (Pre‑training)** → 4. **지도 학습 기반 미세 조정 (Supervised Fine‑tuning)** → 5. **인간 피드백을 활용한 강화 학습 (RLHF)** → 6. **배포 및 추론**  

각 단계가 차례대로 모델이 “언어를 이해하고 생성하는 능력”을 점진적으로 키워 줍니다.

---

## 2. 핵심 기술 – Transformer 아키텍처

| 구성 요소 | 역할 | 핵심 아이디어 |
|-----------|------|----------------|
| **입력 토큰** | 텍스트를 정수 시퀀스로 변환 | Byte‑Pair Encoding (BPE) 혹은 **SentencePiece** 같은 서브워드 토크나이저 사용 |
| **임베딩(Embedding)** | 토큰을 고정 차원의 벡터로 매핑 | 학습 가능한 **토큰 임베딩** + **위치 임베딩** (문장 내 순서를 알려 줌) |
| **멀티‑헤드 셀프‑어텐션** | 각 토큰이 문맥 전체와 상호작용 | `Q, K, V` 행렬을 만든 뒤 `softmax(QKᵀ / √d)` 로 가중치를 구하고, 이를 `V`와 곱해 새로운 표현을 얻음 |
| **피드‑포워드 네트워크(FFN)** | 비선형 변환 | 두 개의 선형 레이어 사이에 `GELU` 같은 활성화 함수 적용 |
| **잔차 연결 + 레이어 정규화** | 학습 안정성·깊은 네트워크 학습 가능 | `x + Sublayer(x)` 형태로 스킵 연결 후 `LayerNorm` 적용 |
| **스택된 레이어** | 모델 깊이 → 표현력 확대 | GPT‑4 같은 최신 모델은 수백 개 레이어와 수천억 파라미터 보유 |

이 구조가 **Causal (autoregressive) Language Model** 로 동작

#### 3) ChatPromptTemplate
* SystemMessagePromptTemplate와 HumanMessagePromptTemplate 클래스 사용
* 객체 지향적 접근 - Message 객체를 독립적으로 생성 가능
* 여러 조건에 따라 다른 시스템 메시지 선택

```python
if user_is_beginner:
    system_message = SystemMessagePromptTemplate.from_template("초보자를 위한 설명: {topic}")
else:
    system_message = SystemMessagePromptTemplate.from_template("전문가를 위한 상세 분석: {topic}")
```

In [15]:
# ChatMessagePromptTemplate 활용

from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    ChatMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "You are an AI expert in {topic}. Please provide clear and detailed explanations."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="What is deep learning?")

# LLM 호출
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

# 결과 출력
print(response.content)

## Deep Learning – A Clear, Detailed Overview

### 1. What is deep learning?
Deep learning is a **subfield of machine learning** that models high‑level abstractions in data by using **artificial neural networks with many (deep) layers**.  
In simple terms, it’s a way for computers to automatically learn useful representations—from raw input (like pixels, audio waveforms, or text) all the way up to complex concepts (objects, speech, language meaning)—without the need for handcrafted features.

| Aspect | Traditional Machine Learning | Deep Learning |
|--------|------------------------------|---------------|
| **Feature engineering** | Manual, domain‑specific | Learned automatically by the network |
| **Model complexity** | Shallow (e.g., linear models, decision trees) | Very deep (tens to thousands of layers) |
| **Data requirement** | Works with relatively small datasets | Thrives on massive datasets |
| **Performance** | Good for structured/tabular data | State‑of‑the‑art on images, s

#### ChatMessagePromptTemplate는 여러 종류의 메시지(시스템, 인간, AI)를 조합하여 복잡한 프롬프트를 생성할 때 유용합니다.
* SystemMessagePromptTemplate: 이 템플릿은 AI 모델에게 역할을 부여하거나 전반적인 규칙을 설정하는 시스템 메시지를 만듭니다. 위의 예시에서는 "번역을 도와주는 유용한 도우미"라는 역할을 지정합니다.
* HumanMessagePromptTemplate: 이 템플릿은 사용자의 질문이나 요청을 담는 인간 메시지를 만듭니다. 아래의 예시에서는 번역할 텍스트를 입력받습니다.
* ChatPromptTemplate.from_messages: 이 클래스 메서드는 시스템 메시지, 인간 메시지 등 여러 종류의 MessagePromptTemplate 객체들을 리스트로 받아 하나의 채팅 프롬프트 템플릿으로 통합합니다.
* format_messages: 이 메서드는 정의된 템플릿에 실제 값을 채워 넣어 [SystemMessage, HumanMessage] 형태의 리스트를 반환합니다. 이 리스트는 채팅 모델(Chat Model) 에 바로 전달될 수 있습니다.

In [16]:
# 필요한 라이브러리 임포트
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 1. SystemMessagePromptTemplate와 HumanMessagePromptTemplate 생성
# SystemMessagePromptTemplate는 모델의 페르소나 또는 기본 지침을 설정합니다.
system_template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# HumanMessagePromptTemplate는 사용자로부터 받는 입력 프롬프트를 정의합니다.
human_template = "{text_to_translate}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# 2. ChatPromptTemplate 생성
# 위에서 만든 두 템플릿을 리스트로 묶어 ChatPromptTemplate을 만듭니다.
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# 3. 프롬프트 포맷팅
# chat_prompt_template.format_messages()를 사용하여 최종 메시지 리스트를 생성합니다.
# 이 함수는 딕셔너리 형태의 입력 변수를 받습니다.
formatted_prompt = chat_prompt_template.format_messages(
    input_language="English",
    output_language="Korean",
    text_to_translate="I love programming."
)

# 4. 결과 출력
print(formatted_prompt)

# LLM 호출
response = llm.invoke(formatted_prompt)

# 결과 출력
print(response.content)

[SystemMessage(content='You are a helpful assistant that translates English to Korean.', additional_kwargs={}, response_metadata={}), HumanMessage(content='I love programming.', additional_kwargs={}, response_metadata={})]
나는 프로그래밍을 사랑합니다.


##### 4) FewShotPromptTemplate
* FewShotPromptTemplate은 모델이 특정 형식을 따르게 하거나, 일관된 응답을 생성하도록 유도할 때 유용합니다.
* 도메인 지식이 필요하거나, AI가 오답을 줄이고 더 신뢰할 만한 답변을 생성하도록 해야 할 때 효과적입니다.

##### 4-1) PromptTemplate을 사용하지 않는 경우

In [19]:
# PromptTemplate을 사용하지 않는 경우
from langchain_openai import ChatOpenAI

# model
#llm = ChatOpenAI(model="gpt-3.5-turbo")

# chain 실행
result = llm.invoke("태양계의 행성들을 간략히 정리해 주세요.")

print(type(result))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
태양계에는 8개의 행성이 있습니다. 

1.  수성: 태양과 가장 가까운 행성으로, 표면이 암석으로 구성되어 있고 극도로 높은 온도와 낮은 온도가 반복됩니다.
2.  금성: 태양계에서 두 번째로 가까운 행성으로, 두꺼운 대기로 인해 극심한 온실 효과가 발생하여 매우 뜨겁습니다.
3.  지구: 우리가 사는 행성으로, 물과 대기가 있어 생명체가 존재할 수 있습니다.
4.  화성: 태양계에서 네 번째로 가까운 행성으로, 붉은색의 모래사막으로 덮여 있고 물과 생명체의 존재 가능성이 있습니다.
5.  목성: 태양계에서 가장 큰 행성으로, 가스 거인이며 강력한 자기장과 수많은 위성을 가지고 있습니다.
6.  토성: 태양계에서 두 번째로 큰 행성으로, 가스 거인이며 아름다운 고리를 가지고 있습니다.
7.  천왕성: 태양계에서 일곱 번째로 가까운 행성으로, 가스 거인이며 자전축이 기울어져 있어 극단적인 기후 변화를 경험합니다.
8.  해왕성: 태양계에서 가장 먼 행성으로, 가스 거인이며 강한 바람과 극적인 기후 변화를 가지고 있습니다.

이러한 행성들은 각각 고유한 특징과 성질을 가지고 있으며, 태양계에서 중요한 역할을 합니다.


##### 4-2) FewShotChatMessagePromptTemplate 사용하는 경우

In [20]:
# FewShotChatMessagePromptTemplate 사용하는 경우
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
chain = final_prompt | llm
print(chain)

# 테스트 실행
# result = chain.invoke({"input": "태양계의 행성들을 간략히 정리해 주세요."})
result = chain.invoke({"input": "양자컴퓨터 정리해 주세요."})
print(result.content)

first=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다.'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': '뉴턴의 운동 법칙을 요약해 주세요.', 'output': '### 뉴턴의 운동 법칙\n1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.\n2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.\n3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다.'}, {'input': '지구의 대기 구성 요소를 알려주세요.', 'output': '### 지구 대기의 구성\n- **질소 (78%)**: 대기의 대부분을 차지합니다.\n- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.\n- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.\n- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다.'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[

#### 5-1) PartialPrompt 
* 프롬프트를 더 동적으로 활용할 수 있으며, AI 응답을 더 일관성 있게 조정 가능함

In [17]:
from datetime import datetime
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# 프롬프트 템플릿 정의 (부분 변수 적용)
prompt = PromptTemplate(
    template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}이 맞나요? {season}에 주로 발생하는 지구과학 현상을 3개 알려주세요",
    input_variables=["phenomenon"],  # 사용자 입력 필요
    partial_variables={"season": get_current_season()}  # 동적으로 계절 값 할당
)

# OpenAI 모델 초기화
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

# 특정 계절의 현상 질의
query = prompt.format(phenomenon="태풍 발생")
result = llm.invoke(query)


# 결과 출력
print(f" 프롬프트: {query}")
print(f" 모델 응답: {result.content}")

 프롬프트: 가을에 일어나는 대표적인 지구과학 현상은 태풍 발생이 맞나요? 가을에 주로 발생하는 지구과학 현상을 3개 알려주세요
 모델 응답: ### 가을에 주로 나타나는 대표적인 지구과학 현상

| # | 현상 | 주요 특징 및 발생 메커니즘 | 가을에 두드러지는 이유 |
|---|------|---------------------------|------------------------|
| 1 | **태풍(열대 저기압) 발생·접근** | • 바다 표면 온도가 ≥ 26 °C 인 열대·아열대 해역에서 발달 <br>• 중심 저기압이 강화되면서 강풍·폭우를 동반 <br>• 이동 경로는 대기 중 고기압·저기압 배치에 크게 좌우 | • 8‑10월은 남서·동남아시아·서태평양 해수면 온도가 가장 높아 ‘태풍 최빈기’가 됩니다.<br>• 동아시아(한반도·중국·일본 등)에서는 태풍이 북상하면서 가을에 가장 많이 영향을 미칩니다. |
| 2 | **한랭전선(Cold Front) 통과와 급격한 기온·습도 변화** | • 남쪽에서 온난·습한 공기가 북쪽에서 차가운 대륙성 공기와 만나면서 형성 <br>• 전선이 통과하면 기온이 급락하고, 바람은 남동·남서 → 북동·북서 방향으로 바뀝니다.<br>• 전선 앞뒤로 구름·비·때로는 눈까지 동반 | • 여름이 끝나면서 북극·시베리아 고기압이 강화되고, 남쪽의 온난 해양 기단이 약화됩니다.<br>• 이때 한랭전선이 남하해 한반도·동아시아에 자주 도달해 ‘가을 기온 급락’ 현상을 일으킵니다. |
| 3 | **가을철 집중호우·홍수·산사태 위험 증가** | • 태풍·전선이 동반하는 폭우가 짧은 시간에 대량 강수 <br>• 산악 지역에서는 급경사면에 물이 급히 모여 **산사태·흙사태**를 유발 <br>• 저지대·하천 주변에서는 급격한 수위 상승으로 **홍수** 위험이 커짐 | • 가을은 태풍·전선이 동시에 작용하는 시기라 강우 강도가 높아집니다.<br>• 토양이 여름 동안 이미 포화 상태에 가까워 있어 배수가 어려워지고, 그 결과 산사태·홍수 위

In [24]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# Step 1: 현재 계절 결정
season_name = get_current_season("south")  # 계절 값 얻기
print(f"현재 계절: {season_name}")


현재 계절: 봄


In [26]:

# Step 2: 해당 계절의 자연 현상 추천
prompt2 = ChatPromptTemplate.from_template(
    "{season}에 주로 발생하는 대표적인 지구과학 현상 3가지를 알려주세요. "
    "각 현상에 대해 간단한 설명을 포함해주세요."
)

# # OpenAI 모델 사용
# #llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
# llm = ChatOpenAI(
#     #api_key=OPENAI_API_KEY,
#     base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
#     model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
#     temperature=0.0
# )

# 체인 2: 자연 현상 추천 (입력: 계절 → 출력: 자연 현상 목록)
chain2 = (
    {"season": lambda x : season_name}  # chain1의 출력을 season 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

# 실행: 현재 계절에 따른 자연 현상 추천
response = chain2.invoke({})
print(f"\n {season_name}에 발생하는 자연 현상:\n{response}")


 봄에 발생하는 자연 현상:
봄에 자주 발생하는 대표적인 지구과학 현상은 다음과 같습니다.

1.  **춘분**: 매년 3월 20일 또는 21일 경에 발생하는 춘분은 낮과 밤의 길이가 거의 같아지는 날입니다. 이는 지구의 축이 태양의 중심을 통과하는 날로, 지구상의 모든 지역에서 낮과 밤의 길이가 거의 같아집니다.

2.  **황사**: 황사는 중국 북서지방의 사막이나 황화지대에서 강한 바람에 의해 일어나는 사막 먼지입니다. 이 현상은 특히 봄철에 자주 발생합니다.

3.  **번개**: 봄철에는 날씨 변화가 심해 번개가 자주 발생합니다. 봄은 대기가 불안정해지면서 상승 기류가 강해지고, 구름이 형성되어 번개가 발생하기 쉽습니다.


#### 5-2) PartialPrompt 
* API 호출 데이터, 시간 정보, 사용자 정보 등을 반영할 때 매우 유용함

In [27]:
import requests
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 실시간 환율을 가져오는 함수
def get_exchange_rate():
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
    data = response.json()
    return f"1달러 = {data['rates']['KRW']}원"

# Partial Prompt 활용
prompt = PromptTemplate(
    template="현재 {info} 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.",
    input_variables=[],  # 사용자 입력 없음
    partial_variables={"info": get_exchange_rate()}  # API에서 가져온 데이터 자동 반영
)

# LLM 모델 설정
#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# 모델에 프롬프트 전달 및 응답 받기
response = llm.invoke(prompt.format())

# 결과 출력
print(" 프롬프트:", prompt.format())
print(" 모델 응답:", response.content)

 프롬프트: 현재 1달러 = 1377.98원 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.
 모델 응답: ## 최근 원/달러 환율 동향 

원/달러 환율은 2023년 1377.98원(2023년 10월 6일 16시 기준)을 기록하고 있습니다. 이는 최근 글로벌 경제의 불확실성과 미국의 통화정책 등 여러 가지 요인에 의해 영향을 받고 있습니다. 

### 주요 요인

*   **미국 경제와 통화 정책**: 미국 경제가 견조한 성장세를 보이고, 인플레이션이 높은 수준을 유지함에 따라 연방준비제도(Fed)는 금리 인상 기조를 유지하고 있습니다. 
*   **글로벌 경제 불확실성**: 글로벌 경제의 불확실성이 지속되며 안전자산 선호도가 높아지고 있습니다. 특히, 주요국들의 경제 성장 둔화와 지정학적 리스크가 원/달러 환율에 영향을 미치고 있습니다. 
*   **원유 가격**: 국제 유가의 변동도 원/달러 환율에 영향을 주는 요인 중 하나입니다. 유가 상승 시, 경상수지 악화 우려로 인해 원화 가치가 하락할 수 있습니다. 

### 전망 

원/달러 환율의 전망은 다양한 변수에 따라 달라질 수 있습니다. Fed의 추가 금리 인상 여부, 글로벌 경제의 회복 속도, 지정학적 이슈 등 여러 요인들이 복합적으로 작용할 것으로 예상됩니다. 

현재로서는 원/달러 환율이 1,350원대 후반에서 등락할 가능성이 있지만, 향후 경제 지표 발표와 글로벌 경제 상황 변화에 따라 변동성이 확대될 수 있습니다. 

### 투자 전략 

*   **환율 변동 위험 관리**: 수출입 기업들은 환율 변동에 따른 위험을 관리하기 위해 환헤지 상품을 활용할 수 있습니다. 
*   **다양한 자산 투자**: 투자자들은 원화 자산과 외화 자산에 분산 투자하여 환율 변동 위험을 줄일 수 있습니다. 
*   **경제 지표 주목**: 향후 경제 지표 발표와 글로벌 경제 상황을 주기적으로 모니터링하여 투자 전략을 조정하는 것이 중요합니다. 

이러한 정보를 바탕으로 투자 및 환율 관련 전